A notebook which contains timeseries of SalishSeaCast model output and observed data within PugetSound for the year 2009. This script is designed so that changes the the observed and model data over time can be easily compared.

In [2]:
import sys
sys.path.append('/ocean/kflanaga/MEOPAR/analysis-keegan/notebooks/Tools')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import pandas as pd
import netCDF4 as nc
import datetime as dt
from salishsea_tools import evaltools as et, viz_tools
import gsw 
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.dates as mdates
import cmocean as cmo
import scipy.interpolate as sinterp
import pickle
import cmocean
import json
import f90nml
import xarray as xr
import datetime as dt
import Keegan_eval_tools as ket
from collections import OrderedDict

fs=16
mpl.rc('xtick', labelsize=fs)
mpl.rc('ytick', labelsize=fs)
mpl.rc('legend', fontsize=fs)
mpl.rc('axes', titlesize=fs)
mpl.rc('axes', labelsize=fs)
mpl.rc('figure', titlesize=fs)
mpl.rc('font', size=fs)
mpl.rc('font', family='sans-serif', weight='normal', style='normal')

import warnings
#warnings.filterwarnings('ignore')
from IPython.display import Markdown, display

%matplotlib inline

In [4]:
year=2010
modelversion='nowcast-green.201905'
PATH= '/results2/SalishSea/nowcast-green.201905/'
datadir='/ocean/eolson/MEOPAR/obs/WADE/ptools_data/ecology'

In [5]:
display(Markdown('''## Year: '''+ str(year)))

## Year: 2010

In [6]:
display(Markdown('''### Model output: '''+ PATH))

### Model output: /results2/SalishSea/nowcast-green.201905/

In [7]:
##### Loading in pickle file data
saveloc='/ocean/kflanaga/MEOPAR/savedData'

with open(os.path.join(saveloc,f'data_WADE_{modelversion}_{year}.pkl'),'rb') as hh:
    data=pickle.load(hh)
    
with open(os.path.join(saveloc,f'data_Pheo_{modelversion}_{year}.pkl'),'rb') as hh:
    data_Pheo=pickle.load(hh)

FileNotFoundError: [Errno 2] No such file or directory: '/ocean/kflanaga/MEOPAR/savedData/data_Pheo_nowcast-green.201905_2010.pkl'

In [ ]:
def TsByRegion(datreg,regions,obsvar,modvar,year,loc='lower left',units='($\mu$M)'):
    fig,ax=plt.subplots(math.ceil(len(regions)/2),2,figsize=(13,13))
    new_reg = [regions[i:i+2] for i in range(0, len(regions), 2)]
    for ri,axi in zip(new_reg,ax):
        for rj,axj in zip(ri,axi):
            ps=ket.tsertser_graph(axj,datreg[rj],obsvar,modvar,dt.datetime(year,1,1),dt.datetime(year,12,31))
            axj.legend(handles=ps,prop={'size': 10},loc=loc)
            axj.set_xlabel(f'Date',fontsize=13)
            axj.set_ylabel(f'{obsvar} {units}',fontsize=13)
            axj.set_title(f'Time series for {rj}', fontsize=13)
            yearsFmt = mdates.DateFormatter('%d %b')
            axj.xaxis.set_major_formatter(yearsFmt)
            for tick in axj.xaxis.get_major_ticks():
                tick.label.set_fontsize(13)
            for tick in axj.yaxis.get_major_ticks():
                tick.label.set_fontsize(13)
            plt.tight_layout()
            plt.setp(axj.get_xticklabels(), rotation=30, horizontalalignment='right')

def TsByDepth(df,obsvar,modvar,year,title,units='($\mu$M)'):
    fig,ax=plt.subplots(1,1,figsize=(18,8))
    ps=ket.tsertser_graph(ax,df,obsvar,modvar,dt.datetime(year,1,1),dt.datetime(year,12,31))
    ax.legend(handles=ps,bbox_to_anchor=[1,.6,0,0])
    ax.set_xlabel(f'Date',fontsize=20)
    ax.set_ylabel(f'{obsvar} {units}',fontsize=20)
    ax.set_title(f'{title}', fontsize=22)
    yearsFmt = mdates.DateFormatter('%d %b')
    ax.xaxis.set_major_formatter(yearsFmt)

# I know I found a solution to this problem of mine in the past that     
    
def TsByStation(df,region,datstat,obsvar,modvar,year):
    for s,a in df[df.Basin == region].Station.unique():
        fig,ax=plt.subplots(1,1,figsize=(18,8))
        ps=ket.tsertser_graph(ax,datstat[s],obsvar,modvar,dt.datetime(year,1,1),dt.datetime(year,12,31))
        ax.legend(handles=ps,bbox_to_anchor=[1,.6,0,0])
        ax.set_xlabel(f'Date',fontsize=20)
        ax.set_ylabel(f'{obsvar} ($\mu$M)',fontsize=20)
        ax.set_title(f'{s}', fontsize=22)
        yearsFmt = mdates.DateFormatter('%d %b')
        ax.xaxis.set_major_formatter(yearsFmt)

In [ ]:
cm1=cmocean.cm.thermal
with nc.Dataset('/data/eolson/results/MEOPAR/NEMO-forcing-new/grid/bathymetry_201702.nc') as bathy:
    bathylon=np.copy(bathy.variables['nav_lon'][:,:])
    bathylat=np.copy(bathy.variables['nav_lat'][:,:])
    bathyZ=np.copy(bathy.variables['Bathymetry'][:,:])

In [ ]:
fig, ax = plt.subplots(1,1,figsize = (6,6))
with nc.Dataset('/data/vdo/MEOPAR/NEMO-forcing/grid/bathymetry_201702.nc') as grid:
    viz_tools.plot_coastline(ax, grid, coords = 'map',isobath=.1)
colors=('blue','green','firebrick','darkorange','darkviolet','fuchsia',
        'royalblue','darkgoldenrod','mediumspringgreen','deepskyblue')
datreg=dict()
for ind, iregion in enumerate(data.Basin.unique()):
    datreg[iregion] = data.loc[data.Basin==iregion]
    ax.plot(datreg[iregion]['Lon'], datreg[iregion]['Lat'],'.',
            color = colors[ind], label=iregion)
ax.set_ylim(47, 49)
ax.legend(bbox_to_anchor=[1,.6,0,0])
ax.set_xlim(-124, -122);
ax.set_title('Observation Locations');

In [ ]:
#creating new dictionaries that make it easy to call on specific years.
datstat=dict()
for ind, istation in enumerate(data.Station.unique()):
    datstat[istation]=data.loc[data.Station == istation]
    
datreg=dict()
for iregion in data.Basin.unique():
    datreg[iregion] = data.loc[data.Basin==iregion] 
    
datyear=dict()
for ind, iyear in enumerate(data.dtUTC.dt.year.unique()):
    datyear[iyear] = data.loc[data.dtUTC.dt.year==iyear]
    
#Adding extra columns to assist in chlorophyll analysis.
data_Pheo['log_Chl']=ket.logt(data_Pheo['Chla_Lab'])
data_Pheo['log_mod_diatoms']=ket.logt(data_Pheo['mod_diatoms']*1.8)
data_Pheo['log_mod_ciliates']=ket.logt(data_Pheo['mod_ciliates']*1.8)
data_Pheo['log_mod_flagellates']=ket.logt(data_Pheo['mod_flagellates']*1.8)
data_Pheo['mod_Chl']=((data_Pheo['mod_diatoms']*1.8)+(data_Pheo['mod_ciliates']*1.8)+(data_Pheo['mod_flagellates']*1.8))
data_Pheo['log_mod_Chl']=ket.logt((data_Pheo['mod_diatoms']*1.8)+(data_Pheo['mod_ciliates']*1.8)+(data_Pheo['mod_flagellates']*1.8))

# defning a dictionary that brakes the data down by individual stations
datstat_Pheo=dict()
for ind, istation in enumerate(data_Pheo.Station.unique()):
    datstat_Pheo[istation]=data_Pheo.loc[data_Pheo.Station == istation]

datreg_Pheo=dict()
for iregion in data_Pheo.Basin.unique():
    datreg_Pheo[iregion] = data_Pheo.loc[data_Pheo.Basin==iregion] 
    
datyear_Pheo=dict()
for iyear in data_Pheo.dtUTC.dt.year.unique():
    datyear_Pheo[iyear] = data_Pheo.loc[data_Pheo.dtUTC.dt.year==iyear]

# Nitrate
## Observed nitrate and model nitrate over time with depth

In [ ]:
obsvar='NO23'
modvar='mod_nitrate'

ket.multi_timese_graph(data,year,obsvar,modvar,(16,8))

## Observed nitrate and model nitrate over time for different regions

In [ ]:
TsByRegion(datreg,data.Basin.unique(),obsvar,modvar,year)

## Observed nitrate and model nitrate over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data[data.Z < 15],obsvar,modvar,year,title)

title='Time series for greater than 22 metres depth'
TsByDepth(data[data.Z > 22],obsvar,modvar,year,title)

## Model Error over time

In [ ]:
ket.multi_timerror_graph(data,datyear,year,obsvar,modvar,(16,9))

# Silicon
## Observed Silicon and model Silicon over time with depth

In [ ]:
obsvar='Si'
modvar='mod_silicon'

ket.multi_timese_graph(data,year,obsvar,modvar,(16,8))

## Observed Silicon and model Silicon over time for different regions

In [ ]:
TsByRegion(datreg,data.Basin.unique(),obsvar,modvar,year)

## Observed Silicon and model Silicon over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data[data.Z < 15],obsvar,modvar,year,title)

title='Time series for greater than 22 metres depth'
TsByDepth(data[data.Z > 22],obsvar,modvar,year,title)

Interesting, for below 22 metres there is a clearly a seasonal pattern to the observed silicon levels, but the modeled silicon levels stay almost completely constant. 

## Model Error over time

In [ ]:
ket.multi_timerror_graph(data,datyear,year,obsvar,modvar,(16,9))

# Ammonium

## Observed Ammonium and model Ammonium over time by 

In [ ]:
obsvar='Amm'
modvar='mod_ammonium'

ket.multi_timese_graph(data,year,obsvar,modvar,(16,8))

## Observed Ammonium and model Ammonium over time for different regions

In [ ]:
TsByRegion(datreg,data.Basin.unique(),obsvar,modvar,year,loc='upper left')

## Observe Ammonium and model Ammonium over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data[data.Z < 15],obsvar,modvar,year,title)

title='Time series for greater than 22 metres depth'
TsByDepth(data[data.Z > 22],obsvar,modvar,year,title)

# Error vs time of year

In [ ]:
ket.multi_timerror_graph(data,datyear,year,obsvar,modvar,(16,9))

# Salinity

## Observed Salinity and model Salinity over time 

In [ ]:
obsvar='SA'
modvar='mod_vosaline'

ket.multi_timese_graph(data,year,obsvar,modvar,(16,8),units='(g/kg)')

## Observed Salinity and model Salinity over time for different regions

In [ ]:
TsByRegion(datreg,data.Basin.unique(),obsvar,modvar,year,loc='upper left',units='(g/kg)')

## Observe Salinity and model Salinity over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data[data.Z < 15],obsvar,modvar,year,title,units='(g/kg)')

title='Time series for greater than 22 metres depth'
TsByDepth(data[data.Z > 22],obsvar,modvar,year,title,units='(g/kg)')

# Error vs time of year

In [ ]:
ket.multi_timerror_graph(data,datyear,year,obsvar,modvar,(16,9),units='(g/kg)')

# Temperature

## Observed Temperature and model Temperature over time 

In [ ]:
obsvar='CT'
modvar='mod_votemper'

ket.multi_timese_graph(data,year,obsvar,modvar,(16,8),units='(C$^o$)')

## Observed Temperature and model Temperature over time for different regions

In [ ]:
TsByRegion(datreg,data.Basin.unique(),obsvar,modvar,year,loc='upper left',units='(C$^o$)')

## Observe Temperature and model Temperature over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data[data.Z < 15],obsvar,modvar,year,title,units='(C$^o$)')

title='Time series for greater than 22 metres depth'
TsByDepth(data[data.Z > 22],obsvar,modvar,year,title,units='(C$^o$)')

# Error vs time of year

In [ ]:
ket.multi_timerror_graph(data,datyear,year,obsvar,modvar,(16,9),units='(C$^o$)')

# Log transformed Chlorophyll
## Observed Chlorophyll and model chlorophyll over time with depth

In [ ]:
obsvar='log_Chl'
modvar='log_mod_Chl'

ket.multi_timese_graph(data_Pheo,year,obsvar,modvar,(16,8))

## Observed nitrate and model nitrate over time for different regions

In [ ]:
obsvar='log_Chl'
modvar='log_mod_Chl'

TsByRegion(datreg_Pheo,data.Basin.unique(),obsvar,modvar,year,loc='upper left')

## Observed nitrate and model nitrate over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data_Pheo[data_Pheo.Z < 15],obsvar,modvar,year,title)

title='Time series for greater than 22 metres depth'
TsByDepth(data_Pheo[data_Pheo.Z > 22],obsvar,modvar,year,title)

## Model Error over time

In [ ]:
ket.multi_timerror_graph(data_Pheo,datyear_Pheo,year,obsvar,modvar,(16,9))

# Chlorophyll
## Observed Chlorophyll and model chlorophyll over time with depth

In [ ]:
obsvar='Chla_Lab'
modvar='mod_Chl'

ket.multi_timese_graph(data_Pheo,year,obsvar,modvar,(16,8))

## Observed Temperature and model nitrate over time for different regions

In [ ]:
TsByRegion(datreg_Pheo,data.Basin.unique(),obsvar,modvar,year,loc='upper left')

## Observed nitrate and model nitrate over time individual depths

In [ ]:
title='Time series for less than 15 metres depth'
TsByDepth(data_Pheo[data_Pheo.Z < 15],obsvar,modvar,year,title)

title='Time series for greater than 22 metres depth'
TsByDepth(data_Pheo[data_Pheo.Z > 22],obsvar,modvar,year,title)

## Model Error over time

In [ ]:
ket.multi_timerror_graph(data_Pheo,datyear_Pheo,year,obsvar,modvar,(16,9))